In [4]:
!pip install -q openai pandas tqdm cleantext

In [50]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm

YOUR_OPENROUTER_KEY = "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"

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key= YOUR_OPENROUTER_KEY
)

In [69]:
PERSIAN_STOPWORDS = {
    "شرکت","موسسه","گروه","صنعت","صنایع","توسعه","مهندسی","فناوری","نوین",
    "تک","ارتباط","مبین","پیشرفته","گسترش","مرکز","هولدینگ",
    "مدرن","نو","جدید","پژوهش","کاربردی","راهکار","راه","راه‌حل",
    "اندیشه","سامانه","خدمات","تجارت","تجاری","بازرگانی","کو","ایران",
   "و", "در", "با", "از",
} 

In [70]:
from shekar import Normalizer, Lemmatizer, WordTokenizer
import re
from cleantext import clean

normalizer = Normalizer()
lemmatizer = Lemmatizer()
tokenizer = WordTokenizer()

def preprocess_and_stem(text): 
    text = normalizer.normalize(text) 
    text = re.sub(r"[^\w\s]", " ", text)
    text = text.replace("\u200c", "")
    text = str(clean(text,
                           clean_all= False  ,
                           extra_spaces=True ,     
                           numbers=True ,  
                           punct=True      
                           ))
     
    tokens = list(tokenizer(text))
     
    stems =  [lemmatizer(t) for t in tokens if t not in PERSIAN_STOPWORDS]
    return " ".join(stems)

In [35]:
preprocess_and_stem("تجارت پارس پويا شمال")

'پارس پویا شمال'

In [36]:
import pandas as pd
data = pd.read_csv('data_sample.csv', header=None, names=['id', 'name'])
data = data.drop(data.index[0])
data = data.reset_index(drop=True)
data.head()

,id,name
0,6C08073088A84609A36859D777FDA74C,كالا پخش عصر ايلام
1,D14E57BFCC2A46438127DB0E9D44E53D,تك ناهيد آدان
2,1F602F123D314279816445A72D249AE8,آراد پارس پويا شمال
3,1DF530F7C9404386B1048F885A9F3CEF,شفق الكتريك هزاره سوم
4,D9A3EB25A9804B679DF5DD3BE7E212E0,امامزاده بي بي بانو روستاي بيان(قير)


In [37]:
registered_names = data['name']
registered_names[:5]

0                      كالا پخش عصر ايلام
1                           تك ناهيد آدان
2                     آراد پارس پويا شمال
3                   شفق الكتريك هزاره سوم
4    امامزاده بي بي بانو روستاي بيان(قير)
Name: name, dtype: object

In [38]:
DATA_GEN_PROMPT = """
شما وظیفه دارید برای هر نام شرکت، نمونه‌هایی بسازید که طبق قوانین زیر مشابه یا غیرمشابه هستند.

قوانین مشابهت (label=1):
1- نام عیناً تکراری باشد.
2- زیرمجموعه‌ی نام باشد یا نام ورودی زیرمجموعه‌ای از ثبت‌شده باشد.
3- جایگشت کلمات نام باشد.
4- تفاوت جزئی املایی یا حذف یک پسوند → مشابه است.
5- جمع و مفرد فرقی ندارد.
6- کلمات مشابه معنایی ("تاجر" و "تجارت") مشابه محسوب شوند.

قوانین غیرمشابه (label=0):
- اگر از نظر معنی، ساختار، حوزه، یا ترکیب کلمات متفاوت باشند.
- نام کاملاً جداگانه باشد.
- شباهت سطحی نداشته باشد.

حداقل 5 نمونه برای هر شرکت تولید کن.
 توضیح اضافی ممنوع.
"""

In [61]:
import json

def generate_pairs_for_name(base_name):
    stemmed_name = preprocess_and_stem(base_name)

    prompt = f"""
    نام ثبت‌شده: "{stemmed_name}"
    براساس قوانین، 3 نمونه مشابه (label=1) و 3 نمونه غیرمشابه (label=0)
    تولید کن.
    """ + DATA_GEN_PROMPT 
    response = client.chat.completions.create(
        model="tngtech/deepseek-r1t-chimera:free",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500,
        temperature=0.3,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "pairs",
                "schema": {
                    "type": "object",
                    "properties": {
                        "items": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "name1": {"type": "string"},
                                    "name2": {"type": "string"},
                                    "label": {"type": "integer"}
                                },
                                "required": ["name1", "name2", "label"]
                            }
                        }
                    },
                    "required": ["items"]
                }
            }
        }
    )

    try: 
        data = json.loads(response.choices[0].message.content)
         
        return data
    except Exception as e:
        print("JSON Error:", e)
        print(response.choices[0].message.content)
        return []

In [62]:
rows = generate_pairs_for_name(registered_names[0])

In [63]:
rows

{'items': [{'name1': 'کالا پخش عصر ایلام',
   'name2': 'کالا پخش عصر ایلام',
   'label': 1},
  {'name1': 'کالا پخش عصر ایلام', 'name2': 'پخش کالا ایلام عصر', 'label': 1},
  {'name1': 'کالا پخش عصر ایلام', 'name2': 'کالاپخش عصر ایلام', 'label': 1},
  {'name1': 'کالا پخش عصر ایلام', 'name2': 'عصر ایلام کالا پخش', 'label': 1},
  {'name1': 'کالا پخش عصر ایلام', 'name2': 'کالا پخش عصار ایلام', 'label': 1},
  {'name1': 'کالا پخش عصر ایلام',
   'name2': 'شرکت تجهیزات پزشکی تهران',
   'label': 0},
  {'name1': 'کالا پخش عصر ایلام', 'name2': 'ایلام کالا', 'label': 0},
  {'name1': 'کالا پخش عصر ایلام', 'name2': 'پخش محصولات کشاورزی', 'label': 0},
  {'name1': 'کالا پخش عصر ایلام', 'name2': 'عصر مدرن ایلام', 'label': 0},
  {'name1': 'کالا پخش عصر ایلام', 'name2': 'مجتمع تجاری کالا', 'label': 0}]}

In [ ]:
from tqdm import tqdm

all_rows = []

for name in tqdm(registered_names[:200]):
    rows = generate_pairs_for_name(name)
    all_rows.extend(rows['items'])



 24%|██▍       | 48/200 [10:43<33:57, 13.40s/it]


APIStatusError: Error code: 402 - {'error': {'message': 'This request requires more credits, or fewer max_tokens. You requested up to 1200 tokens, but can only afford 1131. To increase, visit https://openrouter.ai/settings/credits and upgrade to a paid account', 'code': 402, 'metadata': {'provider_name': None}}, 'user_id': 'user_35pJrycV9yKMRKHPyq09t9Z2PVO'}

In [74]:
len(all_rows)

568

In [75]:

for name in tqdm(registered_names[57:200]):
    rows = generate_pairs_for_name(name)
    all_rows.extend(rows['items'])



  2%|▏         | 3/143 [00:58<45:16, 19.40s/it]

JSON Error: Unterminated string starting at: line 11 column 15 (char 1225)
{"items": [
    {"name1": "مسکن کارکنان کانون پرورش فکری کودکان نوجوانان مراغه", "name2": "مسکن کارکنان کانون پرورش فکری کودکان نوجوانان مراغه", "label": 1},
    {"name1": "مسکن کارکنان کانون پرورش فکری کودکان و نوجوانان مراغه", "name2": "مسکن کارکنان کانون پرورش فکری کودکان نوجوانان مراغه", "label": 1},
    {"name1": "کانون پرورش فکری کودکان نوجوانان مراغه مسکن کارکنان", "name2": "مسکن کارکنان کانون پرورش فکری کودکان نوجوانان مراغه", "label": 1},
    {"name1": "مسکن کارکنان کانون پرورش فکری کودکان نوجوانان مراغا", "name2": "مسکن کارکنان کانون پرورش فکری کودکان نوجوانان مراغه", "label": 1},
    {"name1": "مسکن کارکنان کانون پرورش فکری کودک نوجوان مراغه", "name2": "مسکن کارکنان کانون پرورش فکری کودکان نوجوانان مراغه", "label": 1},
    {"name1": "شرکت ساختمانی مراغه", "name2": "مسکن کارکنان کانون پرورش فکری کودکان نوجوانان مراغه", "label": 0},
    {"name1": "موسسه فرهنگی هنری مراغه", "name2": "مسکن کارکنان کانون پ

TypeError: list indices must be integers or slices, not str

In [53]:
stemmed_name = registered_names.apply(preprocess_and_stem)

In [ ]:
stemmed_name[:10]


0                     کالا پخش عصر ایلام
1                             ناهید آدان
2                    آراد پارس پویا شمال
3                  شفق الکتریک هزاره سوم
4    امامزاده بی بی بانو روستای بیان قیر
5                          پارس تدبیر فن
6                            دهیاری کهنک
7             مدرسه ابتدایی مختلط مولو ی
8                        پرهام سازه ثمین
9                                ساب پوش
Name: name, dtype: object

In [56]:
df= pd.DataFrame(stemmed_name)

In [57]:
df.to_csv("stemmed_dataset.csv", index=False, encoding="utf-8")